Load and prepare data

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/root/endereco_natal.csv")

In [3]:
df.head()

,LOGRADOURO,BAIRRO,CIDADE,ESTADO,CEP
0,Rua 15 de Maio,Pajuçara,Natal,RN,59123-029
1,Travessa 1º de Janeiro,Rocas,Natal,RN,59010-322
2,Rua 25 de Outubro,Pajuçara,Natal,RN,59123-027
3,Vila ABC,Nordeste,Natal,RN,59042-570
4,Vila ABC,Quintas,Natal,RN,59035-218


In [79]:
df_barrios = pd.DataFrame(df['BAIRRO'].unique() , columns=['BARRIOS'])

In [80]:
df.shape
df_barrios.shape
df_barrios

,BARRIOS
0,Pajuçara
1,Rocas
2,Nordeste
3,Quintas
4,Nova Descoberta
5,Neópolis
6,Capim Macio
7,Potengi
8,Nossa Senhora da Apresentação
9,Redinha


Get locate all Neighborhood of Natal RN Brasil

In [81]:
#!pip install geopy

In [82]:
from geopy.geocoders import Nominatim
import webbrowser


In [83]:
geolocator = Nominatim(user_agent="ny_explorer")

def get_lat_log(loc):
    address = loc
    location = geolocator.geocode(address)
    return location

In [84]:
#lt = get_lat_log("Brasil Rio Grande do Norte, Pajuçara, Rua Vinte e Cinco de Outubro")
#print(lt.latitude , lt.longitude)

In [118]:
l = []
lo = []
bar = []

for index, row in df_barrios.iterrows():
    lc = get_lat_log("Brasil, Rio Grande do Norte, Natal, "  + row['BARRIOS']   )
    if lc is not None:
        l.append(lc.latitude)
        lo.append(lc.longitude)
        bar.append( row["BARRIOS"])
    #else:
    #    lc = get_lat_log("Brasil, "  + str(row['CEP']).replace("-" , "").strip() ) 
    #    if lc is not None:
    #        lat = lc.latitude
    #        log = lc.longitude
    


In [121]:
df2=pd.DataFrame(bar,columns=['BAIRRO'])
df2['LATITUDE']=l
df2['LONGITUDE']=lo

In [122]:
df2

,BAIRRO,LATITUDE,LONGITUDE
0,Pajuçara,-5.737235,-35.236043
1,Rocas,-5.773724,-35.199791
2,Nordeste,-5.793003,-35.244401
3,Quintas,-5.797218,-35.231670
4,Nova Descoberta,-5.826920,-35.198547
5,Neópolis,-5.866608,-35.208806
6,Capim Macio,-5.857640,-35.201449
7,Potengi,-5.750582,-35.253447
8,Nossa Senhora da Apresentação,-5.746524,-35.278780
9,Redinha,-5.754051,-35.215259


Get four square api information on Neighborhood

In [153]:

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = 'R04BGUJEVL02E52FMRX0NGFU2NFIYKETP3WXMVTCHMS3G4SY' # your Foursquare ID
CLIENT_SECRET = 'VC51ASWZ20WRPXMAITTB2SSSU0U5YJQS1I2QHZHT1LHPV1QC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
venues_list=[]

for index, row in df2.iterrows():
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        row["LATITUDE"], 
        row["LONGITUDE"], 
        radius, 
        LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    #print(results)
    # return only relevant information for each nearby venue
    venues_list.append([(
        row["BAIRRO"], 
        row["LATITUDE"], 
        row["LONGITUDE"], 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood_Latitude', 
              'Neighborhood_Longitude', 
              'Venue', 
              'Venue_Latitude', 
              'Venue_Longitude', 
              'Venue_Category']    
    


nearby_venues.head()

    

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Pajuçara,-5.737235,-35.236043,Seis E Ponto,-5.747582,-35.247780,Dive Bar
1,Pajuçara,-5.737235,-35.236043,HM Fitness,-5.748366,-35.245207,Gym / Fitness Center
2,Pajuçara,-5.737235,-35.236043,Chiquinho Sorvetes,-5.750725,-35.242752,Ice Cream Shop
3,Pajuçara,-5.737235,-35.236043,Barcana's Petiscaria,-5.744520,-35.247437,Bar
4,Pajuçara,-5.737235,-35.236043,Conveniência 24 horas,-5.741998,-35.247620,Convenience Store


In [175]:
#nearby_venues['Neighborhood'].unique()
#teste = nearby_venues.query('Venue_Category.str.contains("Bus")', engine='python')
#nearby_venues['Venue_Category'].values_count()
pd.set_option('display.max_rows', nearby_venues.shape[0]+1)
print(nearby_venues['Venue_Category'].value_counts())


#nearby_venues.groupby('Venue_Category').count()


Bar                                         214
Bakery                                      188
Gym / Fitness Center                        183
Gym                                         123
Pizza Place                                 110
Dessert Shop                                106
Brazilian Restaurant                         77
Burger Joint                                 75
Acai House                                   72
Pharmacy                                     68
Sandwich Place                               67
Seafood Restaurant                           64
Hot Dog Joint                                62
Restaurant                                   61
Coffee Shop                                  61
Ice Cream Shop                               59
Chocolate Shop                               58
Supermarket                                  57
Northeastern Brazilian Restaurant            56
Beach                                        55
Fast Food Restaurant                    

Filter only bar category venue in dataset

In [187]:
nearby_venues = nearby_venues.query('Venue_Category.str.contains("Bar") and not Venue_Category.str.contains("Salon / Barbershop")', engine='python').drop_duplicates()
nearby_venues

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Pajuçara,-5.737235,-35.236043,Seis E Ponto,-5.747582,-35.247780,Dive Bar
3,Pajuçara,-5.737235,-35.236043,Barcana's Petiscaria,-5.744520,-35.247437,Bar
22,Pajuçara,-5.737235,-35.236043,Barreto's Bar,-5.752689,-35.253757,Bar
52,Pajuçara,-5.737235,-35.236043,Bar do Feijao,-5.765055,-35.262661,Bar
56,Pajuçara,-5.737235,-35.236043,BAR DA JOSY,-5.726890,-35.268477,Bar
66,Pajuçara,-5.737235,-35.236043,Bar Batendo Ponto,-5.769588,-35.256972,Bar
91,Pajuçara,-5.737235,-35.236043,Bar do Camarão,-5.773527,-35.256341,Bar
113,Rocas,-5.773724,-35.199791,Delícias do Mate,-5.786409,-35.205993,Juice Bar
148,Rocas,-5.773724,-35.199791,Beto's Bar,-5.767313,-35.202287,Bar
158,Rocas,-5.773724,-35.199791,Dobradinha Tirol,-5.799669,-35.205135,Bar


Plot point in the map with all bar in Neighborhood of Natal RN Brazil

In [201]:
# create map
l_natal = geolocator.geocode("Natal, RN")

lat_natal = l_natal.latitude
lo_natal = l_natal.longitude

map_clusters = folium.Map(location=[lat_natal, lo_natal], zoom_start=11)

# set color scheme for the clusters
x = np.arange(len(df2))
ys = [i + x + (i*x)**2 for i in range(len(df2))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
n = 0 
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_venues['Venue_Latitude'], nearby_venues['Venue_Longitude'], nearby_venues['Venue'],nearby_venues['Neighborhood']):
    label = folium.Popup(str(poi) + ' Bairro ' + str(cluster), parse_html=True)
    i = str(df2[df2['BAIRRO']==str(cluster)].index.values.astype(int))
    i = i.replace("[","").replace("]","")
    z= int(i)
    #print(i)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[z -1],
        fill=True,
        fill_color=rainbow[z -1],
        fill_opacity=0.7).add_to(map_clusters)
    n = n + 1
       
map_clusters


#import webbrowser
#url = "https://www.google.co.uk/maps/@"+str(latitude)+","+str(longitude)+","+"18z"
#webbrowser.open_new_tab(url)


In [217]:
nearby_venues.groupby('Neighborhood').size().sort_values()



Neighborhood
Guarapes                          1
Areia Preta                       3
Petrópolis                        3
Mãe Luiza                         3
Tirol                             4
Santos Reis                       4
Ribeira                           4
Praia do Meio                     5
Nova Descoberta                   5
Redinha                           5
Rocas                             5
Cidade Alta                       5
Potengi                           5
Igapó                             6
Barro Vermelho                    6
Neópolis                          6
Parque das Dunas                  7
Pajuçara                          7
Lagoa Azul                        7
Cidade Nova                       7
Capim Macio                       7
Ponta Negra                       7
Nossa Senhora da Apresentação     8
Salinas                           8
Lagoa Nova                        9
Planalto                          9
Candelária                        9
Lagoa Seca     

In [216]:
print("Total =====> ", n)


Total =====>  262


The Neighborhood with the best options "bar party", the "Nordest" Neighborhood. But Neighborhood with max potetion busniss to open Bar is Guararapes, Areia Preta, Petropolis, ane Mae Luiza 